# Wander Wise - AI Trip Planner

**User input：**

In [ ]:
location = "Manhattan, New York, NY"
budget = "1000 USD"
days = "5 days"
interests = "Chinese restaurant"

**Install necessary libraries:**

In [ ]:
pip install requests

In [ ]:
pip install openai==0.28.0

In [ ]:
!pip install requests folium
import requests
import folium
from folium.plugins import MarkerCluster

**Return places from Yelp based on inputs**

Option 1 code:

In [ ]:
def get_top_places_from_yelp(location, interests, total_limit=10):
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "Authorization": "..."
    }

    review_threshold = 50

    places = set()

    def fetch_places(term, limit):
        params = {"term": term, "location": location, "limit": limit}
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        if 'businesses' in data:
            for b in data['businesses']:
                if b.get('review_count', 0) >= review_threshold:
                    place = f"{b['name']} ({', '.join(b['location']['display_address'])})"
                    places.add(place)
                    if len(places) >= total_limit:
                        return

    for interest in interests:
        fetch_places(interest, 2)

    general_limit = total_limit - len(places)
    if general_limit > 0:
        fetch_places("tourist attractions", general_limit)

    return list(places)[:total_limit]


Option 2 code

In [ ]:
import requests

def get_top_places_from_yelp(location, interests, total_limit=10):
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "Authorization": "..."
    }

    review_threshold = 30  # Review threshold

    places = set()

    def fetch_places(term, limit):
        params = {"term": term, "location": location, "limit": limit}
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        if 'businesses' in data:
            for b in data['businesses']:
                if b.get('review_count', 0) >= review_threshold:
                    if location.lower() in b['location']['address1'].lower():  # Refined location check
                        place = f"{b['name']} ({', '.join(b['location']['display_address'])})"
                        places.add(place)
                        if len(places) >= total_limit:
                            return

    for interest in interests:
        fetch_places(interest, 1)

    general_limit = total_limit - len(places)
    if general_limit > 0:
        fetch_places("tourist attractions", general_limit)

    return list(places)[:total_limit]


**Save input from Yelp as locations**

In [ ]:
locations = get_top_places_from_yelp(location, interests)
locations

[]

**Use Google Map API to convert locations into coordinates**

In [ ]:
import os
import requests
import folium
from folium.plugins import MarkerCluster

API_KEY = '...'

# Get coordinates for a specific location using Google Maps Geocoding API
def get_coordinates(location):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": location,
        "key": API_KEY
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    if data['status'] == 'OK':
        latlng = data['results'][0]['geometry']['location']
        return latlng['lat'], latlng['lng']
    else:
        print(f"Error fetching coordinates for {location}. Error: {data['status']}")
        return None

for loc in locations:
    coord = get_coordinates(loc)
    if coord:
        print(f"{loc}: {coord}")

**Sort the locations using a greedy algorithm and draw**

In [ ]:
import requests
import folium
from folium.plugins import MarkerCluster

def get_coordinates(location):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": location,
        "key": API_KEY
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    if data['status'] == 'OK':
        latlng = data['results'][0]['geometry']['location']
        return latlng['lat'], latlng['lng']
    else:
        print(f"Error fetching coordinates for {location}. Error: {data['status']}")
        return None

def tsp_greedy(matrix):
    num_locations = len(matrix)
    unvisited = set(range(num_locations))
    path = [0]
    unvisited.remove(0)

    while unvisited:
        last = path[-1]
        next_loc = min(unvisited, key=lambda x: matrix[last][x])
        path.append(next_loc)
        unvisited.remove(next_loc)
    path.append(path[0])
    return path

def get_chunked_distance_matrix(locations):
    max_elements = 10  # Google's limitation is typically 10 for free tier
    num_locations = len(locations)
    matrix = [[0 for _ in range(num_locations)] for _ in range(num_locations)]

    for i in range(0, num_locations, max_elements):
        for j in range(0, num_locations, max_elements):
            chunked_origins = locations[i:i+max_elements]
            chunked_destinations = locations[j:j+max_elements]
            chunk_matrix = get_distance_matrix(chunked_origins, chunked_destinations)

            if chunk_matrix is None:
                print("Error fetching a chunk of distance matrix.")
                return None

            for x, row in enumerate(chunk_matrix):
                for y, distance in enumerate(row):
                    matrix[i+x][j+y] = distance
    return matrix

def get_distance_matrix(origins, destinations=None):
    if destinations is None:
        destinations = origins

    base_url = "https://maps.googleapis.com/maps/api/distancematrix/json"
    params = {
        "origins": "|".join(origins),
        "destinations": "|".join(destinations),
        "key": API_KEY
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    if data['status'] == 'OK':
        matrix = [[row['distance']['value'] for row in entry['elements']] for entry in data['rows']]
        return matrix
    else:
        print(f"Error fetching distance matrix. Error: {data['status']}")
        return None

def plot_with_marker_cluster(locations, tuple_coordinates, base_location):
    base_coord = get_coordinates(base_location)
    m = folium.Map(location=base_coord, zoom_start=13)
    marker_cluster = MarkerCluster().add_to(m)
    for i, (location, coord) in enumerate(zip(locations, tuple_coordinates)):
        if coord:
            folium.Marker(
                location=coord,
                popup=location,
                icon=folium.DivIcon(html=f'<div style="font-size: 12pt; color : black">{i+1}</div>'),
            ).add_to(marker_cluster)

    # Draw polyline without the starting and ending location
    folium.PolyLine(tuple_coordinates, color="blue", weight=2.5, opacity=1).add_to(m)

    return m

full_locations = [location] + locations

coordinates = [get_coordinates(loc) for loc in [location] + locations]

# Using the chunked version to get distance matrix
distance_matrix = get_chunked_distance_matrix(full_locations)
if distance_matrix is None:
    print("Error getting distance matrix. Execution stopped.")
else:
    try:
        optimized_order = tsp_greedy(distance_matrix)
        optimized_coords = [coordinates[i] for i in optimized_order]
        optimized_locations = [full_locations[i] for i in optimized_order]

        # Remove the first and last elements which represent the starting and ending location
        optimized_coords = optimized_coords[1:-1]
        optimized_locations = optimized_locations[1:-1]

        map_object = plot_with_marker_cluster(optimized_locations, optimized_coords, location)
        display(map_object)
    except KeyError as e:
        print(f"An error occurred: {e}")

**Use GPT to generate plans for the arranged locations**

In [ ]:
import openai

def generate_plan(location, budget, days, interest):
    """
    Generate a detailed travel plan based on location, budget, days, and interest using OpenAI API.
    :param location: Location for the travel plan
    :param budget: Budget for the trip
    :param days: Number of days of the trip
    :param interest: Specific interests for the trip
    :return: Generated travel plan
    """
    openai.api_key = "..."

    # Build prompt
    prompt = (
        f"Create a detailed travel plan with activities, meals, and accommodation suggestions.\n"
        f"Location: {location}\n"
        f"Budget: {budget}\n"
        f"Days: {days}\n"
        f"Interest: {interest}\n"
        f"Please provide a daily itinerary with interesting activities including the cost for every activity, recommended restaurants, and suitable places to stay."
    )

    # Invoke the OpenAI API
    response = openai.Completion.create(
        model="text-davinci-003",  # or another suitable model
        prompt=prompt,
        max_tokens=2000  # Adjust as needed
    )

    # Get the generated text
    generated_text = response['choices'][0]['text'].strip()

    return generated_text

# Call the function and print the result
plan = generate_plan(location, budget, days, interests)
print(plan)
